In [1]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import pandas as pd
import torchvision.models as models 
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import matplotlib.pyplot as plt
import latent_plot as lp
from torch.autograd import Variable
from torch.optim import lr_scheduler
import seaborn as sns
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import copy
#from model import VAE

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from data_loader import *
from model import convVAE
from sklearn.manifold import TSNE

In [2]:
#only on validation set
csv_path = '/home/vip/sayan-mandal/datasets/obj_criteria/20200619-objective.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'

ResizeParam = 128
FD = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.RandomHorizontalFlip(p=0.5) ,transforms.ToTensor()]))
FD2 = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.ToTensor()]))

#random sampler
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(FD)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
val_indices =  indices[:split]

# Creating PT data samplers and loaders:
valid_sampler = SubsetRandomSampler(val_indices)

#train_loader = torch.utils.data.DataLoader(FD, batch_size=batch_size, sampler=train_sampler)
data_loader = torch.utils.data.DataLoader(FD2, batch_size=batch_size, sampler=valid_sampler)

In [3]:
#vae model structure
nl = 64
model = convVAE((3,128,128),nl).cuda()
model.load_state_dict(torch.load('/home/vip/sayan-mandal/vae-convmodels/dfcvae_inp128_nl'+str(nl)+'_a1b1.torch'))
print(model)

convVAE(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(64, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Sequential(
    (0): Linear(in_features=16384, out_features=4096, bias=True)
    (1): BatchNo

In [4]:
for i, (data,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10) in enumerate(data_loader):
    y, _,_ =  model.encode(Variable(data).cuda())
    y = y.cpu().detach().numpy()
    if i == 0:
        yd = y
        lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10 = l1.numpy(),l2.numpy(),l3.numpy(),l4.numpy(),l5.numpy(),l6.numpy(),l7.numpy(),l8.numpy(),l9.numpy(),l10.numpy()
    else:
        yd = np.append(yd,y,axis=0)
        lbl1 = np.append(lbl1,l1.numpy(), axis=0)
        lbl2= np.append(lbl2,l2.numpy(), axis=0)
        lbl3= np.append(lbl3,l3.numpy(), axis=0)
        lbl4= np.append(lbl4,l4.numpy(), axis=0)
        lbl5= np.append(lbl5,l5.numpy(), axis=0)
        lbl6= np.append(lbl6,l6.numpy(), axis=0)
        lbl7= np.append(lbl7,l7.numpy(), axis=0)
        lbl8= np.append(lbl8,l8.numpy(), axis=0)
        lbl9= np.append(lbl9,l9.numpy(), axis=0)
        lbl10 = np.append(lbl10,l10.numpy(), axis=0)

In [5]:
yd.shape

(1725, 64)

In [6]:
lbl1 = lbl1.reshape(-1,1)
lbl2 = lbl2.reshape(-1,1)
lbl3 = lbl3.reshape(-1,1)
lbl4 = lbl4.reshape(-1,1)
lbl5 = lbl5.reshape(-1,1)
lbl6 = lbl6.reshape(-1,1)
lbl7 = lbl7.reshape(-1,1)
lbl8 = lbl8.reshape(-1,1)
lbl9 = lbl9.reshape(-1,1)
lbl10 = lbl10.reshape(-1,1)

In [7]:
cols = ['class','mdprob','psdprob','md_inf','md_sup','rnfl_g','rnfl_ti','rnfl_ni','rnfl_ts','rnfl_ns']
for i in range(yd.shape[1]):
    cols += ['ls_'+str(i)]

In [8]:
to_df = np.concatenate((lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10,yd),axis=1)

In [9]:
df = pd.DataFrame(data=to_df, columns=cols)

In [10]:
df

,class,mdprob,psdprob,md_inf,md_sup,rnfl_g,rnfl_ti,rnfl_ni,rnfl_ts,rnfl_ns,...,ls_54,ls_55,ls_56,ls_57,ls_58,ls_59,ls_60,ls_61,ls_62,ls_63
0,1.0,4.0,5.0,2.0,0.0,2.0,2.0,0.0,1.0,1.0,...,-0.312521,-0.000666,0.285729,0.110068,-0.008065,0.102690,0.218448,0.273025,0.532382,-0.312450
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.117288,-0.126811,-0.000619,0.109642,0.230750,0.272240,-0.040699,0.440407,0.173828,-0.013333
2,1.0,5.0,5.0,4.0,4.0,2.0,2.0,0.0,2.0,0.0,...,-0.088064,-0.010177,0.039213,-0.105073,-0.464175,0.165550,-0.085636,-0.486083,0.184655,0.122371
3,0.0,3.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.003085,0.340534,0.184257,-0.344518,0.000413,-0.457679,-0.179798,0.334319,0.014013,-0.048084
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.120189,-0.053361,0.002549,0.081047,0.074008,0.132922,-0.024197,-0.184508,0.065893,0.053854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.134965,0.009408,-0.208372,-0.118267,0.111946,0.184245,0.199419,0.077554,0.102339,0.229039
1721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.017197,0.063621,-0.208461,0.278269,0.035245,-0.191154,-0.037291,-0.075631,-0.166058,-0.197916
1722,1.0,5.0,5.0,4.0,4.0,0.0,2.0,0.0,0.0,0.0,...,0.292122,-0.107703,0.033369,-0.129890,0.117992,0.343188,-0.162703,0.123009,-0.177615,0.134538
1723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.168297,-0.051984,-0.241973,0.068978,-0.117036,-0.083507,-0.025199,0.517580,0.146653,-0.223473


In [11]:
df.to_csv('latentdat_64.csv',index=False)